In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

from datetime import date, datetime, timedelta

In [ ]:
infilename = 'tabula-schedule-of-classes-3.csv'

#data = np.loadtxt(infilename,unpack=True,dtype=str,delimiter=',')
df = pd.read_csv(infilename)

In [ ]:
print(df.shape)
df.columns

In [ ]:
df = df.loc[df['COURSE'] != 'Course']
print(df.shape)

In [ ]:
df['COURSE']

names = []

for val in df['COURSE']:
    names.append(val[0:4])
#print(names)

departments = np.unique(names)
print(departments)

df['Department'] = names

In [ ]:
start_times = []
end_times = []
duration = []

for val in df['Time']:
    sdt = None
    edt = None
    offset = 0
    
    if val.find('-')<0 or val=='Arranged':
        start_times.append(datetime(2021,1,1,1,0))
        end_times.append(datetime(2021,1,1,2,0))
        duration.append(-1)
        continue
    
    s = val.split('-')[0]
    sh,sm = 0,0
    if s.find(':')>=0:
        sh,sm = s.split(':')
        sh = int(sh)
        if sm[-1]=='p':
            offset = 12
            if sh!=12:
                sh += offset
        sm = int(sm[0:2])

        
    e = val.split('-')[1]
    eh,em = 0,0
    if e.find(':')>=0:
        eh,em = e.split(':')
        eh = int(eh)
        em = int(em[0:2])
        
        if eh<sh:
            offset = 12

        if eh != 12:
            eh += offset
        
    #print(s,sh,sm)
    sdt = datetime(2021,1,1,sh,sm)
    edt = datetime(2021,1,1,eh,em)
    dur = edt-sdt
    
    start_times.append(sdt)
    end_times.append(edt)
    duration.append(dur.total_seconds()/3600.)

start_times = np.array(start_times)
end_times = np.array(end_times)
duration = np.array(duration)

df['Start'] = start_times
df['End'] = end_times
df['Duration'] = duration

In [ ]:
end_times[0],start_times[0]

In [ ]:
x = end_times - start_times

plt.figure()
plt.hist(duration,bins=8,range=(0,4))


In [ ]:
df[df['Department']=='PHYS']

In [ ]:
df[df['COURSE']=='MATH120']

In [ ]:
df[df['CRN']=='40151']

In [ ]:
days_dict = {'M':1, 'T':2, 'W':3, 'R':4, 'F':5,'S':6}

colors_dict = {}
colors_dict['MATH'] = 'b'
colors_dict['PHYS'] = 'r'
colors_dict['APHY'] = 'y'
colors_dict['FYSM'] = 'orange'




def display_schedule(dframe=None, selections=None):
    
    fontsize=8

    plt.figure(figsize=(12,12))

    nrows = len(dframe)
    
    for i in range(nrows):
        s = dframe['Start'].iloc[i]
        dur = dframe['Duration'].iloc[i]

        days = dframe['Days'].iloc[i]
        name = dframe['COURSE'].iloc[i]
        section = dframe['Section'].iloc[i]
        instructor = dframe['Instructor'].iloc[i]
        title = dframe['Title'].iloc[i]
        
        department = dframe['Department'].iloc[i]
        color = colors_dict[department]
        
        if dframe['Time'].iloc[i]=='Arranged':
            days = 'S'
            s = datetime(2021,1,1,7,0)
            dur = 3

        #print(name,section,instructor,days,dur)
        for day in days:
            ec = 'k'
            fc = color
            linewidth = 4
            linestyle = '-'
            if title.find('Lab') >= 0:
                linestyle = '--'
            day_num = days_dict[day]-0.5
            #print(day_num)
            plt.gca().add_patch(Rectangle((day_num, s), 1, timedelta(hours=dur), 
                                          alpha=0.1, facecolor=color,ec=ec,lw=linewidth,linestyle=linestyle))
            plt.text(day_num+0.2,s+timedelta(minutes=20),name,fontsize=fontsize)
            plt.text(day_num+0.2,s+timedelta(minutes=40),instructor,fontsize=fontsize)
            plt.text(day_num+0.2,s+timedelta(minutes=60),title,fontsize=fontsize)


    # Play around with the following lines so you see what they do
    plt.gca().yaxis.set_major_locator(mdates.HourLocator(interval=1))
    plt.gca().yaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

    #plt.xticks(rotation=30);
    xlabels = ["S", "M", "T", "W", "R", "F","Arranged"]

    plt.gca().set_xticklabels(xlabels)

    plt.xlim(0,7)
    plt.ylim(datetime(2021,1,1,6,30),datetime(2021,1,1,22,30))
    plt.gca().invert_yaxis()

    
    
#c = df[df['CRN']=='46792']
c = df[df['CRN']=='42697']

#selections = []
c = df[(df['COURSE']=='MATH120') | (df['COURSE']=='PHYS140') | (df['Department']=='APHY') | (df['Title']=='Heroes')]
display_schedule(c)

In [ ]:
end_times[0].hour

In [ ]:
df['Start'][0].hour

In [ ]:
df.size


In [ ]:
df.drop_duplicates()
df.size